In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
import time
import random
import shutil

In [2]:
# Image Data Generator

num_classes = 2

image_resize = 224

batch_size_training = 32
batch_size_validation = 32

batch_size = batch_size_training

num_epochs = 5
learning_rate = 0.01

# Classes

class_1 = "cat"
class_2 = "dog"

# Dataset

train_percent = 0.80
val_percent = 1 - train_percent

# Seeds

np.random.seed(42)
torch.manual_seed(42)

# TensorBoard and MLflow Logging

def create_experiment(class_1, class_2):
    experiment_name = f"{class_1}_{class_2}_classification"
    return experiment_name

def create_runname(model_name):
    run_name = datetime.now().strftime(f"{model_name}_%Y_%m_%d__%H%M%S")
    return run_name

In [3]:
experiment_name = create_experiment(class_1, class_2)
print(experiment_name)

cat_dog_classification


In [4]:
current_directory = os.getcwd()
image_path = "datasets/images"

In [5]:
train_dir = os.path.join(image_path, 'train')
val_dir = os.path.join(image_path, 'validation')

for directory in [train_dir, val_dir]:
    os.makedirs(os.path.join(directory, class_1), exist_ok=True)
    os.makedirs(os.path.join(directory, class_2), exist_ok=True)

def move_images(source_dir, dest_dir, num_images):
    images = os.listdir(source_dir)
    random.shuffle(images)
    for image in images[:num_images]:
        src_path = os.path.join(source_dir, image)
        dest_path = os.path.join(dest_dir, image)
        try:
            shutil.move(src_path, dest_path)
        except PermissionError as e:
            print(f"Permission error occurred: {e}. Skipping file: {src_path}")
        else:
            if dest_path.lower().endswith('.png'):
                try:
                    img = Image.open(dest_path)
                    if img.mode != 'RGBA':
                        img = img.convert('RGBA')
                        img.save(dest_path)
                except Exception as e:
                    print(f"Error converting {dest_path}: {e}")

def split_images(image_path, train_dir, val_dir, train_percent, val_percent):
    for category in [class_1, class_2]:
        source_category_dir = os.path.join(image_path, category)
        train_category_dir = os.path.join(train_dir, category)
        val_category_dir = os.path.join(val_dir, category)

        num_images = len(os.listdir(source_category_dir))
        num_train = int(train_percent * num_images)
        num_val = int(val_percent * num_images)

        move_images(source_category_dir, train_category_dir, num_train)
        move_images(source_category_dir, val_category_dir, num_val)

split_images(image_path, train_dir, val_dir, train_percent, val_percent)
print("Images separated into train and validation directories successfully.")

Images separated into train and validation directories successfully.


In [6]:
transform = transforms.Compose([
    transforms.Resize((image_resize, image_resize)),
    transforms.ToTensor()
])

In [7]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

def load_pytorch_data(directory, transform):
    image_paths = []
    labels = []
    for category in [class_1, class_2]:
        category_path = os.path.join(directory, category)
        label = [class_1, class_2].index(category)
        for file in os.listdir(category_path):
            img_path = os.path.join(category_path, file)
            if os.path.isfile(img_path):
                image_paths.append(img_path)
                labels.append(label)
    return CustomDataset(image_paths, labels, transform=transform)

In [8]:
train_dataset = load_pytorch_data(train_dir, transform)
val_dataset = load_pytorch_data(val_dir, transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size_training, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size_validation, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load SqueezeNet model
model = models.squeezenet1_0(pretrained=True)
model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
model.num_classes = num_classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


c:\Users\MEHMET\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\MEHMET\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
def train_model(model, criterion, optimizer, num_epochs):
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")
    pytorch_training_time = time.time() - start_time
    return pytorch_training_time

pytorch_training_time = train_model(model, criterion, optimizer, num_epochs)

c:\Users\MEHMET\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\TiffImagePlugin.py:890: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1/5, Loss: 17.58868664331436
Epoch 2/5, Loss: 0.6931473015785218
Epoch 3/5, Loss: 0.6931473015785218
Epoch 4/5, Loss: 0.6931473015785218
Epoch 5/5, Loss: 0.6931473015785218


In [11]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

pytorch_accuracy = correct / total

print(f"PyTorch training time: {pytorch_training_time} seconds")
print(f"PyTorch accuracy: {pytorch_accuracy}")

PyTorch training time: 3216.8956673145294 seconds
PyTorch accuracy: 0.5


In [12]:
torch.save(model.state_dict(), 'pytorch_squeezenet_model.pth')